<a href="https://colab.research.google.com/github/Kabelo741/Tweeter-sentiment/blob/master/twitter_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Tweeter Sentiment


The problem at hand is to analyse Twitter Airlines sentiment and the solution that will be applied is to build a Neural Language Processing model that will learn using labeled tweeter data to classify new tweets' sentiment its confidence score.


The method that will be used is Transfer learning and a usefull technique of this methis is called ULMFiT. Transfer learning is when you pretrain a model on a separate dataset in order to teach the model to recognise the task it has to perform before finetuning it on the real dataset. ULMFiT is has three stages: 

a) General-domain LM pretraining  

b) target task LM fine-tuning

c) target task classifier fine-tuning

In [0]:
## Importing Libraries
import numpy as np
import sklearn
import keras
import pandas as pd

Importing the pretraining dataset. I will use IMDB movie reviews that are in the keras.datset library. I chose to use the top 20 000 most used words which means the ML will be trained on a library of 20 000 words. The test split is set to 0.25 by default and cannot be changed which means the training set is 15 000 words. The load_data function returns a sequence of indexes instead of words in the library. The words are index acording to how offen they appear in the library.


In [0]:
from keras.datasets import imdb
(x_train, y_train),(x_test, y_test) = imdb.load_data(path='imbd.npz',num_words = 20000)



Here is one of the sequences of indexes

In [0]:
print(x_train[0])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


Getting and cleaning the dataset. I cleaned the dataset to the best of my ability externally. I removed all columns except for the Sentiment and Text column because they are the only relavent ones.


In [0]:
url = 'https://raw.githubusercontent.com/Kabelo741/Tweeter-sentiment/master/Cleaned_Tweets.csv'
dataset = pd.read_csv(url, error_bad_lines=False, sep = ';')
dataset.dropna()
X = dataset.iloc[:,1:2].values
y = dataset.iloc[:,0:1].values

In [47]:
dataset

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...
5,negative,@VirginAmerica seriously would pay $30 a fligh...
6,NaN,NaN
7,positive,"@VirginAmerica yes, nearly every time I fly VX..."
8,neutral,@VirginAmerica Really missed a prime opportuni...
9,positive,"@virginamerica Well, I didn't…but NOW I DO! :-D"


In [45]:
y

array([['neutral'],
       ['positive'],
       ['neutral'],
       ...,
       ['neutral'],
       ['negative'],
       ['neutral']], dtype=object)

# Preprocessing the data

Taking care of catagorical variables

In [0]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y.astype(str))
y = y.reshape(-1,1)

from sklearn.preprocessing import OneHotEncoder

hot = OneHotEncoder(handle_unknown='ignore')
y = hot.fit_transform(y).toarray()

In [55]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# Building the archetecture


Importing libraries

In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

Initialising the RNN

In [0]:
classifier = Sequential()

Adding layers

In [0]:
classifier.add(LSTM(units=50, return_sequences=True, input_shape = x_train.shape))
classifier.add(LSTM(units=50, return_sequences=True)
classifier.add(LSTM(units=50, return_sequences=True)
classifier.add(Dense(units=50, activation = 'relu'))

